# 영어 Word2Vec 만들기

파이썬의 gensim 패키지는 Word2Vec을 지원하고 있어, 손쉽게 단어를 임베딩 벡터로 변환 시킬 수 있음




In [4]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kwon_notebook\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# 훈련 데이터 이해하기

In [5]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x19fff83f688>)

훈련 데이터 파일은 xml문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요함.

# 훈련 데이터 전처리하기

In [11]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF-8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져옴
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통하여 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalized_text]
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [10]:
# 샘플 3개 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


# Word2Vec 훈련시키기

In [14]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, window=5, min_count=5, workers=4, vector_size=100, sg=0)

size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
window = 컨텍스트 윈도우 크기
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
workers = 학습을 위한 프로세스 수
sg = 0은 CBOW, 1은 Skip-gram.

In [23]:
model_result = model.wv.most_similar("boy")
print(model_result)

[('girl', 0.9263961911201477), ('kid', 0.8425902724266052), ('woman', 0.7881336212158203), ('lady', 0.7717506885528564), ('man', 0.7389834523200989), ('baby', 0.7134214639663696), ('brother', 0.7054275274276733), ('mary', 0.7030795216560364), ('sister', 0.6973580121994019), ('guy', 0.6852748394012451)]


# Word2Vec 모델 저장하고 로드하기

In [25]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v') # 모델 로드

model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8530253767967224), ('guy', 0.8228391408920288), ('lady', 0.7428877949714661), ('boy', 0.7389833927154541), ('girl', 0.7336259484291077), ('gentleman', 0.7105548977851868), ('soldier', 0.6973598599433899), ('kid', 0.6953633427619934), ('poet', 0.6674498319625854), ('surgeon', 0.6629380583763123)]
